In [27]:


import numpy as np
import pandas as pd
!pip install librosa
import librosa
import os

# Function to extract features
def extract_features(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    features = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=38).T, axis=0)
    return features

# Path to the WAV file
audio_path = "C:/Users/nmims.student/Downloads/Kimaya's Audio.wav"

# Extract features
features = extract_features(audio_path)

# Save to CSV
df = pd.DataFrame(features).T  # Transpose to make it a single row

Defaulting to user installation because normal site-packages is not writeable


In [41]:
df

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,-318.375763,136.733627,20.810139,45.047409,-4.771146,27.556932,-3.196316,15.037029,2.432315,-2.289933,...,-2.471559,0.50861,-4.296492,-1.692236,-2.544828,-0.938022,-3.195094,-0.604818,-0.066281,-2.088842


In [42]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
# ==============================
# 1️⃣ Load Saved Model & Encoders
# ==============================
model = load_model(r"C:\Users\nmims.student\Downloads\my_trained_hybrid_model_MHA_BILSTM_CNN.h5")
language_encoder = joblib.load(r"C:\Users\nmims.student\Downloads\language_encoder_mha_bilstm_cnn.pkl")
train_feature_names = joblib.load(r"C:\Users\nmims.student\Downloads\train_feature_names_mha_bilstm_cnn.pkl")
scaler = joblib.load(r"C:\Users\nmims.student\Downloads\scaler_mha_bilstm_cnn.pkl")

sahitya=df
sahitya.columns = [f"feature_{i}" for i in range(len(sahitya.columns))]
sahitya_new = scaler.transform(sahitya)
sahitya_reshaped_new = sahitya_new.reshape(sahitya_new.shape[0], sahitya_new.shape[1], 1)
predictions = model.predict(sahitya_reshaped_new)
predictions
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_class_indices
predicted_labels = language_encoder.inverse_transform(predicted_class_indices)
predicted_labels



C:\Users\NMIMS.Student\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\NMIMS.Student\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step


array(['HINDI'], dtype=object)

In [ ]:

language_codes = ["unknown", "hi-IN", "bn-IN", "kn-IN", "ml-IN", "mr-IN", "od-IN", "pa-IN", "ta-IN", "te-IN", "en-IN", "gu-IN"]


In [19]:
predicted_labels[0]

'MARATHI'

In [ ]:
import requests

# Sample: predicted label and dictionary (adjust with your actual values)
language_dict={'HINDI': 'hi-IN', 'MARATHI': 'mr-IN'}

# Validate predicted label
#if predicted_labels[0] not in language_dict:
    #raise ValueError(f"Unknown language label: {predicted_labels[0]}")

# API setup
url = "https://api.sarvam.ai/speech-to-text"
file_path = "/users/kimayashringarpure/Downloads/Kimaya's Audio.wav"
headers = {
    'api-subscription-key': 'ff014cc1-c60d-4304-b097-76e135ab6ac1'
}

# Request payload
payload = {
    'model': 'saarika:v1',
    'language_code': 'hi-IN',
    #'language_code': language_dict[predicted_labels[0]],
    'with_timesteps': 'false'
}

# Upload file
with open(file_path, 'rb') as audio_file:
    files = [
        ('file', (file_path.split('\\')[-1], audio_file, 'audio/wav'))
    ]
    
    response = requests.post(url, headers=headers, data=payload, files=files)

# Response handling
if response.status_code == 200:
    print(f"✅ Transcription success for language {predicted_labels[0]}:\n{response.text}")
else:
    print(f"❌ Failed with status code {response.status_code}: {response.text}")


In [11]:
print(response.text)

{"request_id":"20250406_04579ffb-7051-4a08-8bf3-d7ea35b46d31","transcript":"यह कहानी के लेखक का नाम क्या है?","language_code":"hi-IN"}


In [54]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from PyPDF2.errors import PdfReadError
from googletrans import Translator

# Load environment variables
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

translator = Translator()

def get_pdf_text(pdf_docs):
    """Extract text from uploaded PDF files, handling errors gracefully."""
    text = ""
    for pdf in pdf_docs:
        try:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                text += page.extract_text()
        except PdfReadError:
            st.warning(f"Unable to read the file: {pdf.name}. Skipping...")
        except Exception as e:
            st.warning(f"An error occurred while processing the file: {pdf.name}. Skipping...")
    return text

def get_txt_text(txt_docs):
    """Extract text from uploaded TXT files, handling decoding errors gracefully."""
    text = ""
    for txt_file in txt_docs:
        try:
            file_content = txt_file.read().decode("utf-8")
            text += file_content
        except UnicodeDecodeError:
            st.warning(f"Unable to decode the file: {txt_file.name}. It may not be in UTF-8 format.")
        except Exception as e:
            st.warning(f"An unexpected error occurred while reading the file: {txt_file.name}. Skipping...")
    return text

def get_text_chunks(text):
    """Split text into manageable chunks."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    return text_splitter.split_text(text)

def get_vector_store(text_chunks):
    """Create and save a FAISS index from text chunks."""
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

def get_conversational_chain():
    """Set up the conversational chain with a custom prompt."""
    prompt_template = """
    Answer the question as detailed as possible from the provided context. If the answer is not in
    the provided context, just say, "answer is not available in the context," without guessing.\n\n
    Context:\n{context}?\n
    Question:\n{question}\n
    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    return load_qa_chain(model, chain_type="stuff", prompt=prompt)

def translate_to_english(text):
    """Translate text from any Indian language to English."""
    try:
        translated = translator.translate(text, dest="en")
        return translated.text
    except Exception as e:
        st.warning("Translation failed. Proceeding with original text.")
        return text

def main():
    st.set_page_config("Chatbot for Your Documents")
    st.header("Chatbot for Your Documents")

    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []

    with st.sidebar:
        st.title("Upload Files")
        uploaded_files = st.file_uploader(
            "Select your Pdf or Text files.",
            accept_multiple_files=True,
            type=["pdf", "txt"]
        )
        if uploaded_files:
            with st.spinner("Processing files..."):
                text = ""
                for file in uploaded_files:
                    if file.name.endswith(".pdf"):
                        text += get_pdf_text([file])
                    elif file.name.endswith(".txt"):
                        text += get_txt_text([file])
                if text.strip():
                    text_chunks = get_text_chunks(text)
                    get_vector_store(text_chunks)
                    st.success("Upload successful.")
                else:
                    st.warning("Uploaded files did not contain any extractable text. Please try again.")

    for message in st.session_state.chat_history:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    user_question = st.chat_input("Ask a question in any Indian language")
    if user_question:
        st.session_state.chat_history.append({"role": "user", "content": user_question})

        with st.chat_message("user"):
            st.markdown(user_question)

        translated_question = translate_to_english(user_question)

        try:
            embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
            index_path = "faiss_index"
            if not os.path.exists(os.path.join(index_path, "index.faiss")):
                raise FileNotFoundError(f"FAISS index file not found at {index_path}/index.faiss. Please process the files first.")
            new_db = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
            docs = new_db.similarity_search(translated_question)
            chain = get_conversational_chain()
            response = chain({"input_documents": docs, "question": translated_question}, return_only_outputs=True)
            assistant_response = response["output_text"]
        except FileNotFoundError as e:
            assistant_response = f"Error: {e}"
        except Exception as e:
            assistant_response = f"An unexpected error occurred: {e}"

        with st.chat_message("assistant"):
            st.markdown(assistant_response)
        st.session_state.chat_history.append({"role": "assistant", "content": assistant_response})

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'streamlit'

In [ ]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from PyPDF2.errors import PdfReadError
from googletrans import Translator

# Load environment variables
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

translator = Translator()

def get_pdf_text(pdf_docs):
    """Extract text from uploaded PDF files, handling errors gracefully."""
    text = ""
    for pdf in pdf_docs:
        try:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                text += page.extract_text()
        except PdfReadError:
            st.warning(f"Unable to read the file: {pdf.name}. Skipping...")
        except Exception as e:
            st.warning(f"An error occurred while processing the file: {pdf.name}. Skipping...")
    return text

def get_txt_text(txt_docs):
    """Extract text from uploaded TXT files, handling decoding errors gracefully."""
    text = ""
    for txt_file in txt_docs:
        try:
            file_content = txt_file.read().decode("utf-8")
            text += file_content
        except UnicodeDecodeError:
            st.warning(f"Unable to decode the file: {txt_file.name}. It may not be in UTF-8 format.")
        except Exception as e:
            st.warning(f"An unexpected error occurred while reading the file: {txt_file.name}. Skipping...")
    return text

def get_text_chunks(text):
    """Split text into manageable chunks."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    return text_splitter.split_text(text)

def get_vector_store(text_chunks):
    """Create and save a FAISS index from text chunks."""
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

def get_conversational_chain():
    """Set up the conversational chain with a custom prompt."""
    prompt_template = """
    Answer the question as detailed as possible from the provided context. If the answer is not in
    the provided context, just say, "answer is not available in the context," without guessing.\n\n
    Context:\n{context}?\n
    Question:\n{question}\n
    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    return load_qa_chain(model, chain_type="stuff", prompt=prompt)

def translate_to_english(text):
    """Translate text from any Indian language to English."""
    try:
        translated = translator.translate(text, dest="en")
        return translated.text
    except Exception as e:
        st.warning("Translation failed. Proceeding with original text.")
        return text

def main():
    st.set_page_config("Chatbot for Your Documents")
    st.header("Chatbot for Your Documents")

    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []

    with st.sidebar:
        st.title("Upload Files")
        uploaded_files = st.file_uploader(
            "Select your Pdf or Text files.",
            accept_multiple_files=True,
            type=["pdf", "txt"]
        )
        if uploaded_files:
            with st.spinner("Processing files..."):
                text = ""
                for file in uploaded_files:
                    if file.name.endswith(".pdf"):
                        text += get_pdf_text([file])
                    elif file.name.endswith(".txt"):
                        text += get_txt_text([file])
                if text.strip():
                    text_chunks = get_text_chunks(text)
                    get_vector_store(text_chunks)
                    st.success("Upload successful.")
                else:
                    st.warning("Uploaded files did not contain any extractable text. Please try again.")

    for message in st.session_state.chat_history:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    user_question = st.chat_input("Ask a question in any Indian language")
    if user_question:
        st.session_state.chat_history.append({"role": "user", "content": user_question})

        with st.chat_message("user"):
            st.markdown(user_question)

        translated_question = translate_to_english(user_question)

        try:
            embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
            index_path = "faiss_index"
            if not os.path.exists(os.path.join(index_path, "index.faiss")):
                raise FileNotFoundError(f"FAISS index file not found at {index_path}/index.faiss. Please process the files first.")
            new_db = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
            docs = new_db.similarity_search(translated_question)
            chain = get_conversational_chain()
            response = chain({"input_documents": docs, "question": translated_question}, return_only_outputs=True)
            assistant_response = response["output_text"]
        except FileNotFoundError as e:
            assistant_response = f"Error: {e}"
        except Exception as e:
            assistant_response = f"An unexpected error occurred: {e}"

        with st.chat_message("assistant"):
            st.markdown(assistant_response)
        st.session_state.chat_history.append({"role": "assistant", "content": assistant_response})

if __name__ == "__main__":
    main()